<a href="https://colab.research.google.com/github/alexandergribenchenko/DS_LATAM_Test/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>